# LIBRARY IMPORT

---



In [5]:
!pip install pyvista
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 35.9 MB/s eta 0:00:00


In [1]:
from skimage import data, io
import plotly.express as px
import numpy as np
import copy
import pyremisel as re
import pyconect as pc
import pyreminfa as prf
import pyprim3d as prim

In [2]:
#Example of using the "help" command for the libraries that were created
help(re)

Help on module pyremisel:

NAME
    pyremisel

DESCRIPTION
    Library for identifying the element ID inside the matrix and eliminating
    floating elements using the Union-Find (Disjoint Set) data structure.

FUNCTIONS
    elim_isel(im, MELT, CFP)
        Function for the elimination of floating elements. It receives as parameters
        the matrix in question, the Young’s modulus of the constituent material, and
        the Poisson’s ratio of the material. The output is a matrix with the floating
        elements set to zero and a vector containing the (Young’s modulus, Poisson’s
        ratio) of each element. Empty elements will contain (0, 0).

    id(position, size)
        Function for determining the relationship between the position in the 3D matrix
        and the element ID. The function input is the element position in the matrix in
        the format (z, y, x) and the total matrix size, also in the format (Z, Y, X). The
        output is the corresponding ID.

FILE
    /

# LOADING THE FILE AND VISUALIZATION

In [3]:
#Variable "im" below is a (Z, Y, X) matrix. In this case, if the thickness is 501,
#the length is 546 and the height is 536, the final "im" size is: (501, 536, 546). To
#check the size, use "cte.shape".
im = io.imread('PCL_corte_retangular_binarizado_diario.tif')
fig_orig = px.imshow(im, animation_frame=0, \
                     labels=dict(animation_frame="Profundidade"), \
                     binary_string=True)
fig_orig.show()
size = im.shape
area = size[0]*size[1]*size[2]

# DETERMINATION OF THE PROPERTIES

In [ ]:
#=======================================================================
#MATERIAL PROPERTIES
#=======================================================================
#Young’s modulus and Poisson’s ratio, both properties
#first from the solid framework and then from the cells
PM_EM_PC = [5, 0.3, 0.9e12, 0.499]
#=======================================================================
#STRUCTURE PROPERTIES
#=======================================================================
EZ, EY, EX = size   #The Z axis is related to layers
TAM_VOX = 1e-5      #Voxel size
W, H, L = (EZ * TAM_VOX, EY * TAM_VOX, EX * TAM_VOX )

# REMOVAL OF FLOATING ELEMENTS

In [ ]:
#To call the corresponding function, the first argument is the
#loaded image matrix, the second is the transverse elastic modulus
#of the framework, and the third is the Poisson’s ratio of the framework.
#The function output is a matrix with floating elements set to zero and
#a vector assigning (elastic modulus, Poisson’s ratio) to each element.
#If the space is empty (element equal to zero), the vector entry for that
#element is (0,0). For more information, type "help(re)"
elim_im, material_elementos = re.elim_isel(im, PM_EM_PC[0], PM_EM_PC[1])
#Creation of a vector containing the list (by ID) of the "valid" elements,
#that is, elements belonging to the solid framework. Thus, the IDs of empty
#elements are not included in the list. For more details on how each
#element ID is determined, see "pyremisel.py".
elementos_validos = [ i for i in range(len(material_elementos)) \
                      if material_elementos[i] != (0,0)]
#Plot of the matrix after removing the floating elements
elim_fig = px.imshow(elim_im, animation_frame=0, \
                     labels=dict(animation_frame="Profundidade"),\
                     binary_string=True)
elim_fig.show()
#If desired, save the elimination result to a .tif file. To
#do this, uncomment the line below
#io.imsave('resultado.tif', elim_im)

# FORMATION OF THE CONNECTIVITY AND COORDINATE MATRICES

In [6]:
#The connectivity and coordinate matrices are formed for a structure composed of
#same-sized cubes, filling the entire space defined by W, H and L. The arguments
#are the number of elements in each direction (EZ, EY and EX) and their respective
#sizes. For more details, type help(pc)
mconect, mcoord = pc.create_mcs (EZ, EY, EX, W, H, L)

# FORMATION OF TETRAHEDRAL ELEMENTS WITH SMOOTHING

## REORGANIZATION OF THE FRAMEWORK INTO TRIANGULAR FACES

In [ ]:
#For the use of future libraries, the structure must be in the form
#of triangular faces. Thus, only the elements that contain framework are
#considered in this process, then each face of the original cube is split
#into two triangles. The connectivity matrix generated at the end contains
#each triangle in a separate row.
mconect_polydata = copy.deepcopy(mconect)[elementos_validos]
mconect_polydata -= 1
mconect_all_faces_triangle = [np.stack((face[[0,1,2]], face[[0,2,3]], face[[4,5,6]], \
                                        face[[4,6,7]], face[[1,2,5]], face[[5,2,6]], \
                                        face[[0,3,4]], face[[3,4,7]], face[[0,1,5]], \
                                        face[[0,4,5]], face[[3,2,6]], face[[3,6,7]]))\
                              for face in mconect_polydata]
mconect_all_faces_triangle = np.reshape(mconect_all_faces_triangle,(len(mconect_polydata)*12, -1))

## REMOVAL OF INTERNAL FACES

In [ ]:
#With the way the faces were generated, each internal face will appear
#twice in the face connectivity matrix. Thus, the removal of internal faces
#is done by deleting these faces that repeat 2×, eliminating them
#completely and not only the duplicate occurrence in the matrix. For more details,
#type "help(prf)"
mconect_faces_triangle_shell = prf.elim_infa (mconect_all_faces_triangle)

## FORMAÇÃO DE TETRAETROS COM TRIÂNGULOS INTERNOS E EXTERNOS

In [ ]:
#For generating the mesh with tetrahedral elements, some previous
#steps are performed: removal of floating vertices, removal of
#duplicate faces (1×), mesh subdivision (each subdivision
#splits each face into 4×), and mesh smoothing. To learn more
#about the input and output of the function used, type "help(prim)"
n_subd = 1      #Number of mesh subdivisions
it_smooth = 2   #Number of smoothing iterations
#=======================================================================
#WITH INTERNAL AND EXTERNAL TRIANGLES
#=======================================================================
prim.smooth_tmsh(mcoord, mconect_all_faces_triangle, TAM_VOX, \
                 "malha_final_trimesh_all_faces", n_subd, it_smooth)
prim.smooth_plot ("malha_final_trimesh_all_faces.ply", \
                  "malha_final_trimesh_all_faces_smooth.ply")
#=======================================================================
#WITH SHELL TRIANGLES
#=======================================================================
prim.smooth_tmsh(mcoord, mconect_faces_triangle_shell, TAM_VOX, \
                 "malha_final_trimesh_shell_faces", n_subd, it_smooth)
prim.smooth_plot ("malha_final_trimesh_shell_faces.ply", \
                  "malha_final_trimesh_shell_faces_smooth.ply")